# Lab Topology

![](jncia_dev_topo.png)

## Base config on Olive-1

```
config
    set int em0 unit 0 family inet address 192.168.122.11/24
    set system service ssh
    set system service netconf ssh 
    set system root-authentication plain-text
    Password123
    commit 
exit
```

# SSH Communication

## SSH using paramiko

In [1]:
import paramiko

In [2]:
olive1 = {
    'ip': '192.168.122.11',
    'username':'root',
    'password':'Password123'
}

In [5]:
def exec_command_ssh_param(device, command):
    ssh_session = paramiko.SSHClient()
    ssh_session.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh_session.connect(hostname=device['ip'],
                        username=device['username'],
                        password=device['password'])
    stdin, stdout, stderr = ssh_session.exec_command('cli '+command)

    print('Output(s) \n----------------')
    for line in stdout.readlines():
        print(line)
    
    print('Error(s) \n----------------')
    if len(stderr.readlines()) == 0:
        print('Np Errors')
    else:
        for line in stderr.readlines():
            print(line)
            
    ssh_session.close()

In [32]:
exec_command_ssh_param(device=olive1, command=input('Enter a command... '))

Enter a command... sh int em0
Output(s) 
----------------
Physical interface: em0, Enabled, Physical link is Up

  Interface index: 8, SNMP ifIndex: 17

  Type: Ethernet, Link-level type: Ethernet, MTU: 1514, Speed: 1000mbps

  Device flags   : Present Running

  Interface flags: SNMP-Traps

  Link type      : Full-Duplex

  Current address: 0c:cd:00:f0:2d:00, Hardware address: 0c:cd:00:f0:2d:00

  Last flapped   : 2021-04-18 13:33:48 UTC (02:28:01 ago)

    Input packets : 1080 

    Output packets: 882



  Logical interface em0.0 (Index 69) (SNMP ifIndex 18) 

    Flags: SNMP-Traps Encapsulation: ENET2

    Input packets : 1072 

    Output packets: 882

    Protocol inet, MTU: 1500

      Flags: Sendbcast-pkt-to-re, Is-Primary

      Addresses, Flags: Is-Default Is-Preferred Is-Primary

        Destination: 192.168.122/24, Local: 192.168.122.11, Broadcast: 192.168.122.255

Error(s) 
----------------
Np Errors


## SSH using napalm

In [7]:
import napalm 

In [17]:
def exec_command_ssh_napalm(device, commands):
    driver = napalm.get_network_driver('junos')
    device = driver(hostname = device['ip'],
                    username = device['username'],
                    password = device['password'])
    device.open()
    output = device.cli(commands)
    for key in output.keys():
        print(f'Cmd: {key} \n----------------')
        print(output[key])
        
    device.close()

In [21]:
exec_command_ssh_napalm(device=olive1, commands=['sh ver | disp xml','sh int em0'])

Cmd: sh ver | disp xml 
----------------

Model: olive
JUNOS Base OS boot [12.1R1.9]
JUNOS Base OS Software Suite [12.1R1.9]
JUNOS Kernel Software Suite [12.1R1.9]
JUNOS Crypto Software Suite [12.1R1.9]
JUNOS Packet Forwarding Engine Support (M/T Common) [12.1R1.9]
JUNOS Packet Forwarding Engine Support (M20/M40) [12.1R1.9]
JUNOS Online Documentation [12.1R1.9]
JUNOS Voice Services Container package [12.1R1.9]
JUNOS Border Gateway Function package [12.1R1.9]
JUNOS Services AACL Container package [12.1R1.9]
JUNOS Services LL-PDF Container package [12.1R1.9]
JUNOS Services PTSP Container package [12.1R1.9]
JUNOS Services Stateful Firewall [12.1R1.9]
JUNOS Services NAT [12.1R1.9]
JUNOS Services Application Level Gateways [12.1R1.9]
JUNOS Services Captive Portal and Content Delivery Container package [12.1R1.9]
JUNOS Services RPM [12.1R1.9]
JUNOS Services HTTP Content Management package [12.1R1.9]
JUNOS AppId Services [12.1R1.9]
JUNOS IDP Services [12.1R1.9]
JUNOS Services Crypto [12.1R1.9

# NETCONF

## activating NETCONF on Junos 

* use `set system services netconf ssh` command to activate netconf 
* use `sh system service` command to verify if netconf is running.
* create a netconf a connection from remote system using `ssh -l USERNAME HOST_IP -p 830 -s netcof` command

```
ssh root@192.168.122.11 -p 830 -s netconf

The authenticity of host '[192.168.122.11]:830 ([192.168.122.11]:830)' can't be established.
ECDSA key fingerprint is SHA256:kCVfql44deqeqslENge7mIJ37V1HDccsaV1yhY0ohxw.
Are you sure you want to continue connecting (yes/no/[fingerprint])? yes
Warning: Permanently added '[192.168.122.11]:830' (ECDSA) to the list of known hosts.
root@192.168.122.11's password: 

<!-- No zombies were killed during the creation of this user interface -->
<!-- user root, class super-user -->
<hello>
  <capabilities>
    <capability>urn:ietf:params:xml:ns:netconf:base:1.0</capability>
    <capability>urn:ietf:params:xml:ns:netconf:capability:candidate:1.0</capability>
    <capability>urn:ietf:params:xml:ns:netconf:capability:confirmed-commit:1.0</capability>
    <capability>urn:ietf:params:xml:ns:netconf:capability:validate:1.0</capability>
    <capability>urn:ietf:params:xml:ns:netconf:capability:url:1.0?protocol=http,ftp,file</capability>
    <capability>http://xml.juniper.net/netconf/junos/1.0</capability>
    <capability>http://xml.juniper.net/dmi/system/1.0</capability>
  </capabilities>
  <session-id>2000</session-id>
</hello>
]]>]]>
```

## Interacting with netconf shell 
* get a xml-rpc equevalant call of any given command using `COMMAND | disp xml rpc`
* paste the body enclosed within `<rpc>...</rpc>` tags

```
show interfaces em0 | display xml rpc  

<rpc-reply xmlns:junos="http://xml.juniper.net/junos/12.1R1/junos">
    <rpc>
        <get-interface-information>
                <interface-name>em0</interface-name>
        </get-interface-information>
    </rpc>
    <cli>
        <banner></banner>
    </cli>
</rpc-reply>
```

* Netconf return the following as an output

```
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/12.1R1/junos">
<interface-information xmlns="http://xml.juniper.net/junos/12.1R1/junos-interface" junos:style="normal">
<physical-interface>
<name>
em0
</name>
<admin-status junos:format="Enabled">
up
</admin-status>
<oper-status>
up
</oper-status>
<local-index>
8
</local-index>
<snmp-index>
17
</snmp-index>
<if-type>
Ethernet
</if-type>
<link-level-type>
Ethernet
</link-level-type>
<mtu>
1514
</mtu>
<speed>
1000mbps
</speed>
<if-device-flags>
<ifdf-present/>
<ifdf-running/>
</if-device-flags>
<if-config-flags>
<iff-snmp-traps/>
</if-config-flags>
<link-type>
Full-Duplex
</link-type>
<if-media-flags>
<internal-flags>
0x4
</internal-flags>
</if-media-flags>
<current-physical-address junos:format="MAC 0c:cd:00:f0:2d:00">
0c:cd:00:f0:2d:00
</current-physical-address>
<hardware-physical-address junos:format="MAC 0c:cd:00:f0:2d:00">
0c:cd:00:f0:2d:00
</hardware-physical-address>
<interface-flapped junos:seconds="3082">
2021-04-18 13:33:48 UTC (00:51:22 ago)
</interface-flapped>
<traffic-statistics junos:style="brief">
<input-packets>
605
</input-packets>
<output-packets>
430
</output-packets>
</traffic-statistics>
<logical-interface>
<name>
em0.0
</name>
<local-index>
69
</local-index>
<snmp-index>
18
</snmp-index>
<if-config-flags>
<iff-snmp-traps/>
</if-config-flags>
<encapsulation>
ENET2
</encapsulation>
<traffic-statistics junos:style="brief">
<input-packets>
597
</input-packets>
<output-packets>
430
</output-packets>
</traffic-statistics>
<filter-information>
</filter-information>
<address-family>
<address-family-name>
inet
</address-family-name>
<mtu>
1500
</mtu>
<address-family-flags>
<ifff-is-primary/>
<ifff-sendbcast-pkt-to-re/>
</address-family-flags>
<interface-address>
<ifa-flags>
<ifaf-current-default/>
<ifaf-current-preferred/>
<ifaf-current-primary/>
</ifa-flags>
<ifa-destination>
192.168.122/24
</ifa-destination>
<ifa-local>
192.168.122.11
</ifa-local>
<ifa-broadcast>
192.168.122.255
</ifa-broadcast>
</interface-address>
</address-family>
</logical-interface>
</physical-interface>
</interface-information>
</rpc-reply>
]]>]]>
```

* Althouth the output looks a bit cluttered but it is identical to the interface em0 configuration in xml format, this can be verified using `sh int em0 | disp xml` command. 

```
<rpc-reply xmlns:junos="http://xml.juniper.net/junos/12.1R1/junos">
    <interface-information xmlns="http://xml.juniper.net/junos/12.1R1/junos-interface" junos:style="normal">
        <physical-interface>
            <name>em0</name>
            <admin-status junos:format="Enabled">up</admin-status>
            <oper-status>up</oper-status>
            <local-index>8</local-index>
            <snmp-index>17</snmp-index>
            <if-type>Ethernet</if-type>
            <link-level-type>Ethernet</link-level-type>
            <mtu>1514</mtu>
            <speed>1000mbps</speed>
            <if-device-flags>
                <ifdf-present/>
                <ifdf-running/>
            </if-device-flags>
            <if-config-flags>
                <iff-snmp-traps/>
            </if-config-flags>
            <link-type>Full-Duplex</link-type>
            <if-media-flags>
                <internal-flags>0x4</internal-flags>
            </if-media-flags>           
            <current-physical-address junos:format="MAC 0c:cd:00:f0:2d:00">0c:cd:00:f0:2d:00</current-physical-address>
            <hardware-physical-address junos:format="MAC 0c:cd:00:f0:2d:00">0c:cd:00:f0:2d:00</hardware-physical-address>
            <interface-flapped junos:seconds="3263">2021-04-18 13:33:48 UTC (00:54:23 ago)</interface-flapped>
            <traffic-statistics junos:style="brief">
                <input-packets>619</input-packets>
                <output-packets>444</output-packets>
            </traffic-statistics>       
            <logical-interface>         
                <name>em0.0</name>      
                <local-index>69</local-index>
                <snmp-index>18</snmp-index>
                <if-config-flags>       
                    <iff-snmp-traps/>   
                </if-config-flags>      
                <encapsulation>ENET2</encapsulation>
                <traffic-statistics junos:style="brief">
                    <input-packets>611</input-packets>
                    <output-packets>444</output-packets>
                </traffic-statistics>   
                <filter-information>    
                </filter-information>   
                <address-family>        
                    <address-family-name>inet</address-family-name>
                    <mtu>1500</mtu>     
                    <address-family-flags>
                        <ifff-is-primary/>
                        <ifff-sendbcast-pkt-to-re/>
                    </address-family-flags>
                    <interface-address> 
                        <ifa-flags>     
                            <ifaf-current-default/>
                            <ifaf-current-preferred/>
                            <ifaf-current-primary/>
                        </ifa-flags>    
                        <ifa-destination>192.168.122/24</ifa-destination>
                        <ifa-local>192.168.122.11</ifa-local>
                        <ifa-broadcast>192.168.122.255</ifa-broadcast>
                    </interface-address>
                </address-family>       
            </logical-interface>        
        </physical-interface>           
    </interface-information>            
    <cli>                               
        <banner></banner>               
    </cli>                              
</rpc-reply>            
```

## Interacting with NETCONF using ncclient python library 

* To make the netconf communication more managable and automated `ncclient` library can be used. This helps fetching the raw xml data from a remote machine 
* To make the raw XML data more structured, `xmltodict` and `json` library are used.
    * __`xmltodict`__: Converts raw XML into an Ordered dictionary
    * __`json`__: converts a dictionary object into JSON serialised strucuted

In [22]:
import json
import xmltodict
from ncclient import manager
from ncclient.xml_ import to_ele

In [29]:
def exec_command_netconf(device, xml_rpc):
    # initiating connection 
    with manager.connect(host=device['ip'], 
                         username=device['username'],
                         password=device['password'],
                         port=830) as m:
        temp = m.dispatch(           # fetching responses 
                    to_ele(xml_rpc)) # convert xml string into element tree 
        print(json.dumps(         
                xmltodict.parse(    # parsing xml into order-dict 
                    str(temp)       # converting xml object into string 
                    ), indent=4))   # json formatting with indent 4 spaces

In [30]:
xml_rpc_int_em0 = """<get-interface-information>
                        <interface-name>em0</interface-name>
                     </get-interface-information>"""

In [31]:
exec_command_netconf(device=olive1, xml_rpc=xml_rpc_int_em0)

{
    "rpc-reply": {
        "@xmlns": "urn:ietf:params:xml:ns:netconf:base:1.0",
        "@xmlns:junos": "http://xml.juniper.net/junos/12.1R1/junos",
        "@xmlns:nc": "urn:ietf:params:xml:ns:netconf:base:1.0",
        "@message-id": "urn:uuid:90d30698-91c6-4cbf-93c0-423beb3aab2b",
        "interface-information": {
            "@xmlns": "http://xml.juniper.net/junos/12.1R1/junos-interface",
            "@junos:style": "normal",
            "physical-interface": {
                "name": "em0",
                "admin-status": {
                    "@junos:format": "Enabled",
                    "#text": "up"
                },
                "oper-status": "up",
                "local-index": "8",
                "snmp-index": "17",
                "if-type": "Ethernet",
                "link-level-type": "Ethernet",
                "mtu": "1514",
                "speed": "1000mbps",
                "if-device-flags": {
                    "ifdf-present": null,
                   

# PyEZ
* install Juniper python library using `pip install juniper`
* to convert any RPC calls into PyEZ function call, replace the "-" sybmols to "_"  e.g. "`<rpc><get-interface-information>`" is "`rpc.get_interface_information()`" 
* docs: https://www.juniper.net/documentation/en_US/junos-pyez/information-products/pathway-pages/junos-pyez-developer-guide.pdf

In [61]:
from jnpr.junos import Device
import pprint
import lxml

router1 = Device(host=olive1['ip'],
                 user=olive1['username'],
                 password=olive1['password'],
                 normalize=True,
                 get_facts=True)

In [63]:
with router1.open() as rtr:
    pp=pprint.PrettyPrinter(indent=4)
    pp.pprint(rtr.facts)

{'2RE': False,
 'HOME': '/root',
 'RE0': None,
 'RE1': None,
 'RE_hw_mi': None,
 'current_re': ['re0'],
 'domain': None,
 'fqdn': '',
 'hostname': '',
 'hostname_info': {'re0': ''},
 'ifd_style': 'CLASSIC',
 'junos_info': {'re0': {'object': junos.version_info(major=(12, 1), type=R, minor=1, build=9),
                        'text': '12.1R1.9'}},
 'master': None,
 'model': 'OLIVE',
 'model_info': {'re0': 'OLIVE'},
 'personality': 'OLIVE',
 're_info': None,
 're_master': None,
 'serialnumber': None,
 'srx_cluster': None,
 'srx_cluster_id': None,
 'srx_cluster_redundancy_group': None,
 'switch_style': 'NONE',
 'vc_capable': False,
 'vc_fabric': None,
 'vc_master': None,
 'vc_mode': None,
 'version': '12.1R1.9',
 'version_RE0': '12.1R1.9',
 'version_RE1': None,
 'version_info': junos.version_info(major=(12, 1), type=R, minor=1, build=9),
 'virtual': True}


In [101]:
with router1.open() as rtr:
    print(
                lxml.etree.tostring(                   # sends XML 
                rtr.rpc.get_interface_information(),   # returns xlms etree
                pretty_print=True,                     # pritty print 
                encoding='unicode',                    # string formatted    
                )
        )

<interface-information style="normal">
  <physical-interface>
    <name>cbp0</name>
    <admin-status format="Enabled">up</admin-status>
    <oper-status>up</oper-status>
    <local-index>132</local-index>
    <snmp-index>501</snmp-index>
    <if-type>Ethernet</if-type>
    <link-level-type>Ethernet</link-level-type>
    <mtu>1514</mtu>
    <if-device-flags>
      <ifdf-present/>
      <ifdf-running/>
    </if-device-flags>
    <if-config-flags>
      <iff-snmp-traps/>
    </if-config-flags>
    <link-type>Full-Duplex</link-type>
    <if-media-flags>
      <ifmf-none/>
    </if-media-flags>
    <current-physical-address format="MAC 00:05:86:71:7d:11">00:05:86:71:7d:11</current-physical-address>
    <hardware-physical-address format="MAC 00:05:86:71:7d:11">00:05:86:71:7d:11</hardware-physical-address>
    <interface-flapped seconds="0">Never</interface-flapped>
    <traffic-statistics style="brief">
      <input-packets>0</input-packets>
      <output-packets>0</output-packets>
    </tr

In [84]:
with router1.open() as rtr:
    cmd = input('Enter a command: ')
    print(f'xml rpc:\n {rtr.display_xml_rpc(cmd,format="text")}')

Enter a command: sh version
xml rpc:
 <get-software-information>
</get-software-information>


# Ansible

## Basics

* Configuration management and deployment tools
* Declarative: final state is defined, how the state is reached is automated.
* Idempotent: Multiple time applying same thing, only the first one will work. 
* Clientless: No agent needed on the target
    * Server: copy module over ssh
    * Junos: Execute lically NETCONF RPC (Scripts executes locally on Ansible server)
* Accessing ansible modules 
    * Ansible Core: supported by anisble team, for Juniper it is Junos 
    * Ansible Galaxy: 3rd party and community supported, for juniper it is Juniper.junos

### Components

* Inventory: lists of targets, located in `/etc/ansible/hosts`
    ```
    [firewall]
        192.168.1.1    ansible_user=root    ansible_ssh_pass=Pass123
        192.168.1.2    ansible_user=root    ansible_ssh_pass=Pass123
    [routers]
        192.168.2.1    ansible_user=root    ansible_ssh_pass=Pass123
        192.168.2.2    ansible_user=root    ansible_ssh_pass=Pass123
     ```
* Varibales 
    * inventory 
    * group_vars
    * host_vars
    * custom files
* Playbook
    ```
    ---
    - name : Example
      roles : 
          - Juniper.junos
      hosts : routers
      gather_facts: no
      user: {{ user }}
      tasks:
         - name: Show Interfaces
           juniper_junos_command:
               commands:
                   show interfaces "{{int}}" terse
           register: junos_ints
        - name: PrintINterface
          debug: 
              msg:{{junos_ints.stdout_lines}}
    ```
    

## Installation

* Make sure python3.6+ is installed: `python --version`
* Install Ansible:
 
    ```
    sudo apt -y update 
    sudo apt -y install software-properties-common
    sudo apt add-repository --yes --update ppa:ansible/ansible
    sudo apt -y install ansible
    pip install ansible
    ```
* verify installation: `ansible --version`

    ```
    ansible 2.10.8
      config file = /etc/ansible/ansible.cfg
      configured module search path = ['/home/rishi/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
      ansible python module location = /home/rishi/anaconda3/lib/python3.8/site-packages/ansible
      executable location = /usr/bin/ansible
      python version = 3.8.5 (default, Sep  4 2020, 07:30:14) [GCC 7.3.0]
    ```
* Verify connectivity: `ansible localhost -m ping`

    ```
    localhost | SUCCESS => {
    "changed": false,
    "ping": "pong"
    }
    ```

# REST

## Basic
![](junos_rest.png)

* activate rest : 
    * `set system services rest`
    * `set system services rest enable-explorer`